# Eyepoint RX data pull

# Connection

In [1]:
## importing packages and setting connector to snowflake
import pandas as pd
from drg_connect import Snowflake
pd.options.display.float_format = '{:.0f}'.format
#pd.set_option('display.max_columns', None)
#pd.set_option('display.height', 500)
#pd.set_option('display.max_rows', 500)

import warnings
warnings.filterwarnings('ignore')

In [2]:
##defining parameters of snowflake
snow = Snowflake(role = 'RWD_ANALYTICS_RW', warehouse = 'RWD_WH_ANALYTICS' ,database='PROJECT_ANALYTICS',schema ='EYEPOINT_PHYFACTAR')
engine = snow.engine

%reload_ext sql_magic
%config SQL.output_result = True  #Enable output to std out
%config SQL.notify_result = False #disable browser notifications
%config SQL.conn_name = 'engine'  #Set the sql_magic connection engine

# Drug Refrence table

In [9]:
# Refrenvce table for NSAID/STEROID/OMIDRIA
drug_ref_table = pd.read_excel('nsaid_steroid_list_redbook_post_approval_1608_v3.xlsx',dtype=str)
drug_ref_table.head()

code                         thrclds  prodnme  \
0  00003017222  Antiinflam S/MM Agnts&Comb NEC  KENALOG   
1  00003017268  Antiinflam S/MM Agnts&Comb NEC  KENALOG   
2  00003017270  Antiinflam S/MM Agnts&Comb NEC  KENALOG   
3  00003017280  Antiinflam S/MM Agnts&Comb NEC  KENALOG   
4  00003017360  Antiinflam S/MM Agnts&Comb NEC  KENALOG   

                    gennme                     gnindds drug_type  gen_bran  
0  Triamcinolone Acetonide  Multisource brand, generic   STEROID  Generics  
1  Triamcinolone Acetonide  Multisource brand, generic   STEROID  Generics  
2  Triamcinolone Acetonide  Multisource brand, generic   STEROID  Generics  
3  Triamcinolone Acetonide  Multisource brand, generic   STEROID  Generics  
4  Triamcinolone Acetonide  Multisource brand, generic   STEROID  Generics

In [10]:
%%read_sql
drop table if exists ep_drug_ref_table;

Query started at 01:22:18 PM India Standard Time; Query executed in 0.08 m

status
0  EP_DRUG_REF_TABLE successfully dropped.

In [11]:
snow.upload_dataframe(drug_ref_table,'ep_drug_ref_table', schema='EYEPOINT_PHYFACTAR')

Upload successful!


In [12]:
%%read_sql
select distinct drug_type from ep_drug_ref_table

Query started at 01:22:39 PM India Standard Time; Query executed in 0.06 m

drug_type
0   STEROID
1     NSAID
2       OMI

In [13]:
%%read_sql
select count(distinct code) from ep_drug_ref_table

Query started at 01:22:44 PM India Standard Time; Query executed in 0.05 m

COUNT(DISTINCT CODE)
0                  7575

# Raven Medical

1. table to be used here: Eyepoint_cataract_raven_phy
2. WHERE physician_npi is not null

In [3]:
%%read_sql
--creating index date 
create or replace table EYEPOINT_CATARACT_MIN_DATE_sr as
      select distinct *, min(YEAR_OF_SERVICE) over (partition by PATIENT_ID) as patient_min_date
        from Eyepoint_cataract_raven_phy
       where physician_npi is not null;

Query started at 06:12:44 PM India Standard TimeInitiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
; Query executed in 0.34 m

status
0  Table EYEPOINT_CATARACT_MIN_DATE_SR_1 successf...

In [4]:
%%read_sql

SELECT count(distinct patient_id) as patient_count,
       count(distinct claim_number) as claim_count
  FROM EYEPOINT_CATARACT_MIN_DATE_sr

Query started at 06:14:00 PM India Standard Time; Query executed in 0.32 m

patient_count  claim_count
0         650628      1336420

In [5]:
%%read_sql
create or replace table EYEPOINT_CATARACT_MIN_DATE_nik_3_sr as
       select *,ROW_NUMBER() OVER (PARTITION BY patient_id ORDER BY year_of_Service ASC) AS ROWNUM
       from EYEPOINT_CATARACT_MIN_DATE_sr

Query started at 06:14:35 PM India Standard Time; Query executed in 0.08 m

status
0  Table EYEPOINT_CATARACT_MIN_DATE_NIK_3_SR_1 su...

In [6]:
%%read_sql

SELECT count(distinct patient_id) as patient_count,
       count(distinct claim_number) as claim_count
  FROM EYEPOINT_CATARACT_MIN_DATE_nik_3_sr

Query started at 06:15:07 PM India Standard Time; Query executed in 0.06 m

patient_count  claim_count
0         650628      1336420

## NDC and procedure from the Procedure tables

Isha: We need to join on the patinet id and put a filter for the drugs codes here only

In [7]:
%%read_sql
--Pulling ndc and procedure of cataract claims
CREATE OR REPLACE TABLE EYEPOINT_CATARACT_MED_NDC_sr AS
      SELECT distinct A.PATIENT_ID,
                      B.CLAIM_NUMBER,
                      A.YEAR_OF_SERVICE AS CATARACT_DATE,
                      A.patient_min_date,
                      A.Physician_NPI as Physician_id,
                         
                      B.NDC,
                      B.PROCEDURE,
                      B.YEAR_OF_SERVICE AS DRUG_DATE
        FROM EYEPOINT_CATARACT_MIN_DATE_nik_3_sr A
             LEFT JOIN RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_PROCEDURE B
                       ON A.PATIENT_ID = case when B.ENCRYPTED_KEY_2 is null then B.ENCRYPTED_KEY_1 else B.ENCRYPTED_KEY_1||B.ENCRYPTED_KEY_2 end
                          AND DATEDIFF(day,patient_min_date,B.YEAR_OF_SERVICE) between -30 and 90
              ;

Query started at 06:15:31 PM India Standard Time; Query executed in 13.06 m

status
0  Table EYEPOINT_CATARACT_MED_NDC_SR_1 successfu...

# Pulling Provider Information from Raven

In [16]:
%%read_sql
--DROP TABLE IF EXISTS EYEPOINT_CATARACT_PROC_HEAD_DIAG_PROV;
CREATE  or replace TABLE EYEPOINT_CATARACT_MED_NDC_Prov as
    SELECT A.*,
           B.PROVIDER_TYPE,
           B.PROVIDER_NPI,
           B.ENTITY_TYPE_CODE
  FROM EYEPOINT_CATARACT_MED_NDC_SR  A
       LEFT JOIN RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_PROVIDER B
              ON A.claim_number = B.claim_number
 WHERE LOWER(B.PROVIDER_TYPE) IN ('attending','rendering','operating','facility','billing','servicelocation');

Query started at 05:13:03 PM India Standard Time; Query executed in 8.00 m

status
0  Table EYEPOINT_CATARACT_MED_NDC_PROV successfu...

# Provider Normalisation

In [17]:
%%read_sql

CREATE OR REPLACE TABLE EYEPOINT_CATARACT_MED_NDC_PROV_Nor  as
     SELECT A.*,
            case when b.provider_NPI is null then
            case when c.Provider_NPI is null then
            d.Provider_NPI else c.Provider_NPI end
            else b.provider_NPI end as Physician_NPI
           
           
           
       FROM EYEPOINT_CATARACT_MED_NDC_PROV  A
            LEFT JOIN (select distinct claim_number,Provider_NPI
                         from EYEPOINT_CATARACT_MED_NDC_PROV 
                        where lower(provider_type) = 'attending'
                              AND ENTITY_TYPE_CODE = 1) as b
                   on a.claim_number = b.claim_number
            left join (select distinct claim_number,Provider_NPI
                         from EYEPOINT_CATARACT_MED_NDC_PROV 
                        where lower(provider_type) = 'rendering'
                              AND ENTITY_TYPE_CODE = 1) as c
                   on a.claim_number = c.claim_number
            left join (select distinct claim_number,Provider_NPI
                         from EYEPOINT_CATARACT_MED_NDC_PROV 
                        where lower(provider_type) = 'operating'
                              AND ENTITY_TYPE_CODE = 1) as d
                   on a.claim_number = d.claim_number
            
                   

Query started at 05:22:21 PM India Standard Time; Query executed in 0.25 m

status
0  Table EYEPOINT_CATARACT_MED_NDC_PROV_NOR succe...

## Update Provider table using ISha's Code

In [8]:
%%read_sql
CREATE OR REPLACE transient TABLE EYEPOINT_CATARACT_MED_NDC_PROV_Nor_sr 
AS 
  SELECT * 
  FROM   ( 
                         SELECT DISTINCT patient_id, 
                                         a.classification, 
                                         a.specialty, 
                                         b.npi, 
                                         c.claim_number , 
                                         row_number() over(partition BY patient_id,c.claim_number ORDER BY
                                         CASE 
                                                         WHEN classification='Specialist' THEN 1
                                                         WHEN classification='General Acute Care Hospital' THEN 2
                                                         ELSE 9 
                                         end)AS rnk 
                         FROM            rwd_db.rwd_reference_library.cms_provider_specialty_codes a
                         JOIN            rwd_db.rwd.third_party_cmsnpi b 
                         ON              a.code=b.healthcare_provider_taxonomy_code_1 
                         JOIN            rwd_db.rwd.raven_claims_submits_provider c 
                         ON              cast(b.npi AS VARCHAR)=c.provider_npi 
                         JOIN 
                                         ( 
                                                  SELECT   claim_number, 
                                                           patient_id, 
                                                           count(*) 
                                                  FROM     EYEPOINT_CATARACT_MED_NDC_PROV_Nor 
                                                  GROUP BY 1, 
                                                           2 
                                                  HAVING   count( DISTINCT physician_npi)>1) d 
                         ON              c.claim_number=d.claim_number)a 
  WHERE  rnk=1 ;

Query started at 06:28:34 PM India Standard Time; Query executed in 4.14 m

status
0  Table EYEPOINT_CATARACT_MED_NDC_PROV_NOR_SR su...

In [16]:
%%read_sql
select count(*) from EYEPOINT_CATARACT_MED_NDC_PROV_Nor_sr

Query started at 06:54:06 PM India Standard Time; Query executed in 0.05 m

COUNT(*)
0       260

In [13]:
%%read_sql
CREATE OR REPLACE TABLE EYEPOINT_CATARACT_MED_NDC_PROV_Nor_final AS 
SELECT DISTINCT a.patient_id, 
                a.claim_number, 
                b.CATARACT_DATE,
                b.patient_min_date,
                B.NDC,
                B.PROCEDURE,
                B.DRUG_DATE,
                coalesce(cast(a.npi AS VARCHAR),cast(b.physician_npi AS VARCHAR)) AS physician_npi 
FROM            EYEPOINT_CATARACT_MED_NDC_PROV_Nor_sr a 
RIGHT JOIN      EYEPOINT_CATARACT_MED_NDC_PROV_Nor b 
ON              a.claim_number=b.claim_number 
AND             a.patient_id=b.patient_id; 

Query started at 06:48:54 PM India Standard Time; Query executed in 0.73 m

status
0  Table EYEPOINT_CATARACT_MED_NDC_PROV_NOR_FINAL...

In [25]:
%%read_sql
select distinct claim_number, count(distinct physician_npi) from EYEPOINT_CATARACT_MED_NDC_PROV_Nor_final
 group by 1 having count(distinct physician_npi)>1

Query started at 07:39:24 PM India Standard Time; Query executed in 0.15 m

Empty DataFrame
Columns: [claim_number, COUNT(DISTINCT PHYSICIAN_NPI)]
Index: []

In [14]:
%%read_sql
select count(*),
       count(distinct claim_number) as clm_cnt,
       count(distinct patient_id) as pat_cnt,
       count(distinct physician_npi) as cnts_physician 
  from EYEPOINT_CATARACT_MED_NDC_PROV_Nor_final

Query started at 06:49:55 PM India Standard Time; Query executed in 0.15 m

COUNT(*)  clm_cnt  pat_cnt  cnts_physician
0  23071603  6470039   650628          436832

## Payer info from base tables

###  Adding Payer Info from Vulture

In [17]:
%%read_sql
create or replace table EYEPOINT_CATARACT_MED_NDC_PROV_NOR_payer_vul_sr as
select distinct A.PATIENT_ID,
       A.CLAIM_NUMBER,
       A.CATARACT_DATE,
       A.PATIENT_MIN_DATE,
       A.physician_npi,
       A.NDC,
       A.PROCEDURE,
       A.DRUG_DATE,
       B.FINGERTIP_PARENT_ID,
       B.FINGERTIP_PARENT_NAME,
       B.FINGERTIP_PAYER_ID,
       B.FINGERTIP_PAYER_NAME,
       B.PAYER_ID,
       B.name,
       B.CLAIM_FILE_INDICATOR AS TYPE_COVERAGE
from  EYEPOINT_CATARACT_MED_NDC_PROV_Nor_final  A
left join rwd_db.rwd.vulture_payer B
          on right(A.claim_number,length(A.claim_number)-4) = B.claim_id
where lower(left(A.CLaim_Number, 3)) = 'vul' 
;

Query started at 07:06:32 PM India Standard Time; Query executed in 1.19 m

status
0  Table EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER...

In [18]:
%%read_sql
select count(distinct claim_number) as clm_cnt,
       count(distinct patient_id) as pat_cnt,
       count(distinct physician_npi) as cnts_facility from  EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_vul_sr

Query started at 07:12:17 PM India Standard Time; Query executed in 0.10 m

clm_cnt  pat_cnt  cnts_facility
0  1387454   172354         111705

### Adding Payer Info from Albatross

In [19]:
%%read_sql
create or replace table  EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_alb_sr as
select distinct A.PATIENT_ID,
       A.CLAIM_NUMBER,
       A.CATARACT_DATE,
       A.PATIENT_MIN_DATE,
       A.Physician_NPI,
       A.NDC,
       A.PROCEDURE,
       A.DRUG_DATE,
       B.PRIMARY_FINGERTIP_PARENT_ID,
       B.PRIMARY_FINGERTIP_PARENT_NAME,
       B.PRIMARY_FINGERTIP_PAYER_ID,
       B.PRIMARY_FINGERTIP_PAYER_NAME,
       B.SECONDARY_FINGERTIP_PARENT_ID,
       B.SECONDARY_FINGERTIP_PARENT_NAME,
       B.SECONDARY_FINGERTIP_PAYER_ID,
       B.SECONDARY_FINGERTIP_PAYER_NAME,
       B.TERTIARY_FINGERTIP_PARENT_ID,
       B.TERTIARY_FINGERTIP_PARENT_NAME,
       B.TERTIARY_FINGERTIP_PAYER_ID,
       B.TERTIARY_FINGERTIP_PAYER_NAME,
       B.PRIMARYPAYERID,
       B.PRIMARYPAYERNAME,
       B.SECDPAYERNAME,
       B.SECDPAYERPRIMARYID,
       B.THIRDPAYERNAME,
       B.THIRDPAYERPRIMARYID,
       B.sourceofpayment AS TYPE_COVERAGE
from  EYEPOINT_CATARACT_MED_NDC_PROV_Nor_final A
left join rwd_db.rwd.albatross_claims_header B
          on right(A.claim_number,length(A.claim_number)-4) = cast(B.ENTITYID as varchar())
where lower(left(A.claim_number,3))= 'alb'  
;

Query started at 07:12:53 PM India Standard Time; Query executed in 2.36 m

status
0  Table EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER...

In [20]:
%%read_sql
select count(distinct claim_number) as clm_cnt,
       count(distinct patient_id) as pat_cnt,
      count(distinct physician_npi) as cnts_physician from EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_alb_sr

Query started at 07:19:29 PM India Standard Time; Query executed in 0.15 m

clm_cnt  pat_cnt  cnts_physician
0   606811   149711           26873

### Pulling Payer Information from Condor

In [21]:
%%read_sql
create or replace table EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_con_sr as
select distinct A.PATIENT_ID,
       A.CLAIM_NUMBER,
       A.CATARACT_DATE,
       A.PATIENT_MIN_DATE,
       A.Physician_NPI,
       A.NDC,
       A.PROCEDURE,
       A.DRUG_DATE,
       B.FINGERTIP_PARENT_ID,
       B.FINGERTIP_PARENT_NAME,
       B.FINGERTIP_PAYER_ID,
       B.FINGERTIP_PAYER_NAME,
       B.PAYER_ID,
       B.PAYER_NAME,
       B.TYPE_COVERAGE
from EYEPOINT_CATARACT_MED_NDC_PROV_Nor_final A
left join rwd_db.rwd.condor_claim_record B
          on right(A.claim_number,length(A.claim_number)-4) = B.claim_number
where lower(left(A.claim_number,3))= 'con'
;

Query started at 07:19:55 PM India Standard Time; Query executed in 9.01 m

status
0  Table EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER...

In [22]:
%%read_sql
select count(distinct claim_number) as clm_cnt,
       count(distinct patient_id) as pat_cnt,
       count(distinct Physician_npi) as cnts_Physician_npi from EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_con_sr

Query started at 07:34:00 PM India Standard Time; Query executed in 0.38 m

clm_cnt  pat_cnt  cnts_physician_npi
0  3710555   485065              343454

### Pulling Payer Information from Osprey

In [26]:
%%read_sql
--Mx
create or replace table EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_Osp_sr as
select distinct A.PATIENT_ID,
       A.CLAIM_NUMBER,
       A.CATARACT_DATE,
       A.PATIENT_MIN_DATE,
       A.Physician_NPI,
       A.NDC,
       A.PROCEDURE,
       A.DRUG_DATE,
       B.PAYER_NAM,
       B.MDCL_PAYER_ID,
       B.SRCE_OF_PAY_CDE AS TYPE_COVERAGE
from EYEPOINT_CATARACT_MED_NDC_PROV_NOR_final A
left join rwd_db.rwd.osprey_T_MX_DA0_PAYER1_REC B
          on A.claim_number = cast(B.TXN_GID as varchar())
;         

Query started at 07:39:55 PM India Standard Time; Query executed in 1.05 m

status
0  Table EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER...

In [29]:
%%read_sql
create or replace table EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_Osp_1_sr as
select distinct A.*,
       B.FINGERTIP_PARENT_ID,
       B.FINGERTIP_PARENT_NAME,
       B.FINGERTIP_PAYER_ID,
       B.FINGERTIP_PAYER_NAME
from EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_Osp_sr a
left join rwd_db.rwd.osprey_mx_name_id_addr b
on a.PAYER_NAM = b.PAYER_NAM
and a.MDCL_PAYER_ID = b.MDCL_PAYER_ID
;


Query started at 07:44:49 PM India Standard Time; Query executed in 0.30 m

status
0  Table EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER...

In [24]:
%%read_sql
--HX
create or replace table EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_Osp_2_sr as
select distinct A.PATIENT_ID,
       A.CLAIM_NUMBER,
       A.CATARACT_DATE,
       A.PATIENT_MIN_DATE,
       A.Physician_NPI,
       A.NDC,
       A.PROCEDURE,
       A.DRUG_DATE,
       B.PRCSR_PAYER_NAM as MDCL_PAYER_2_NAM,
       B.MDCL_PAYER_ID,
       B.SRCE_OF_PAY_CDE AS TYPE_COVERAGE
from EYEPOINT_CATARACT_MED_NDC_PROV_NOR_final A
left join RWD_DB.RWD.OSPREY_T_HX_30_3RD_PARTY_PAYER1_REC B
          on A.claim_number = cast(B.TXN_GID as varchar())
;

Query started at 07:36:04 PM India Standard Time; Query executed in 1.01 m

status
0  Table EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER...

In [30]:
%%read_sql
create or replace table EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_Osp_3_sr as
select distinct A.*,
       B.FINGERTIP_PARENT_ID,
       B.FINGERTIP_PARENT_NAME,
       B.FINGERTIP_PAYER_ID,
       B.FINGERTIP_PAYER_NAME
from EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_Osp_2_sr a
left join rwd_db.rwd.osprey_hx_name_id_addr b
on a.MDCL_PAYER_ID = b.MDCL_PAYER_ID
AND a.MDCL_PAYER_2_NAM = b.MDCL_PAYER_2_NAM
;

Query started at 07:45:24 PM India Standard Time; Query executed in 0.51 m

status
0  Table EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER...

In [31]:
%%read_sql
create or replace table EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_Osp_SR_1 as
select * from EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_Osp_1_sr
union
select * from EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_Osp_3_sr
;

Query started at 07:46:18 PM India Standard Time; Query executed in 0.58 m

status
0  Table EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER...

### Union all Payer Tables

In [32]:
%%read_sql
create or replace table EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_Info_sr as
select distinct PATIENT_ID,
       CLAIM_NUMBER,
       CATARACT_DATE,
       PATIENT_MIN_DATE,
       Physician_NPI,
       NDC,
       PROCEDURE,
       DRUG_DATE,
       TYPE_COVERAGE,
       PAYER_ID,
       name,
       FINGERTIP_PARENT_ID,
       FINGERTIP_PARENT_NAME,
       FINGERTIP_PAYER_ID,
       FINGERTIP_PAYER_NAME
  from EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_Vul_sr
 --where FINGERTIP_PAYER_ID is not null
union
select distinct PATIENT_ID,
       CLAIM_NUMBER,
       CATARACT_DATE,
       PATIENT_MIN_DATE,
       Physician_NPI,
       NDC,
       PROCEDURE,
       DRUG_DATE,
       TYPE_COVERAGE,
       coalesce(PRIMARYPAYERID,SECDPAYERPRIMARYID,THIRDPAYERPRIMARYID) as payer_id,
       coalesce(PRIMARYPAYERNAME,SECDPAYERNAME,THIRDPAYERNAME) as payer_name,
       coalesce(PRIMARY_FINGERTIP_PARENT_ID,SECONDARY_FINGERTIP_PARENT_ID,TERTIARY_FINGERTIP_PARENT_ID) as FINGERTIP_PARENT_ID,
       coalesce(PRIMARY_FINGERTIP_PARENT_NAME,SECONDARY_FINGERTIP_PARENT_NAME,TERTIARY_FINGERTIP_PARENT_NAME) as FINGERTIP_PARENT_NAME,
       coalesce(PRIMARY_FINGERTIP_PAYER_ID,SECONDARY_FINGERTIP_PAYER_ID,TERTIARY_FINGERTIP_PAYER_ID) as FINGERTIP_PAYER_ID,
       coalesce(PRIMARY_FINGERTIP_PAYER_NAME,SECONDARY_FINGERTIP_PAYER_NAME,TERTIARY_FINGERTIP_PAYER_NAME) as FINGERTIP_PAYER_NAME
  from EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_alb_sr
 --where coalesce(PRIMARY_FINGERTIP_PAYER_ID,SECONDARY_FINGERTIP_PAYER_ID,TERTIARY_FINGERTIP_PAYER_ID) is not null
union
select distinct PATIENT_ID,
       CLAIM_NUMBER,
       CATARACT_DATE,
       PATIENT_MIN_DATE,
       Physician_NPI,
       NDC,
       PROCEDURE,
       DRUG_DATE,
       TYPE_COVERAGE,
       PAYER_ID,
       PAYER_NAME,
       FINGERTIP_PARENT_ID,
       FINGERTIP_PARENT_NAME,
       FINGERTIP_PAYER_ID,
       FINGERTIP_PAYER_NAME
  from EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_con_sr
  --  where FINGERTIP_PAYER_ID is not null  
union
select distinct PATIENT_ID,
       CLAIM_NUMBER,
       CATARACT_DATE,
       PATIENT_MIN_DATE,
       Physician_NPI,
       NDC,
       PROCEDURE,
       DRUG_DATE,
       TYPE_COVERAGE,
       MDCL_PAYER_ID,
       PAYER_NAM,
       FINGERTIP_PARENT_ID,
       FINGERTIP_PARENT_NAME,
       FINGERTIP_PAYER_ID,
       FINGERTIP_PAYER_NAME
from EYEPOINT_CATARACT_MED_NDC_PROV_Nor_Payer_Osp_SR_1
--where FINGERTIP_PAYER_ID is not null
;

Query started at 07:47:33 PM India Standard Time; Query executed in 0.85 m

status
0  Table EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER...

In [36]:
%%read_sql
select count(*) as counts,
       count(distinct Physician_npi) as cnts_physicians,
       count(distinct patient_id) as patient_counts,
       count(distinct claim_number) as claim_counts
       from EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_Info

Query started at 06:14:44 PM India Standard Time; Query executed in 0.09 m

counts  cnts_physicians  patient_counts  claim_counts
0  40617504           435644          648925       6401645

In [33]:
%%read_sql
select count(*) as counts,
       count(distinct Physician_npi) as cnts_physicians,
       count(distinct patient_id) as patient_counts,
       count(distinct claim_number) as claim_counts
       from EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_Info_sr
       
       --	23071603	6470039	650628	436832

Query started at 07:48:36 PM India Standard Time; Query executed in 0.23 m

counts  cnts_physicians  patient_counts  claim_counts
0  64367849           436832          650628       6470039

## TOC

In [34]:
%%read_sql
CREATE OR REPLACE TABLE EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_TOC_sr AS
      SELECT A.*,
             C.MAP1 as TOC
        FROM EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_Info_sr A
             LEFT Join project_analytics.Pfizer.Plan_Type_Mapping_Options C
                    on A.TYPE_COVERAGE = C.TYPE_OF_COVERAGE;

Query started at 07:53:59 PM India Standard Time; Query executed in 1.26 m

status
0  Table EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER...

In [35]:
%%read_sql

SELECT count(distinct claim_number),
       count(distinct PATIENT_ID)
  from EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_TOC_sr 

Query started at 07:55:46 PM India Standard Time; Query executed in 0.28 m

COUNT(DISTINCT CLAIM_NUMBER)  COUNT(DISTINCT PATIENT_ID)
0                       6470039                      650628

In [36]:
%%read_sql
    SELECT TOC,
           COUNT(DISTINCT PATIENT_ID) AS PAT_CNT,
           COUNT(DISTINCT CLAIM_NUMBER) AS CLM_CNT
      FROM EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_TOC_sr
     GROUP BY TOC
     ORDER BY CLM_CNT DESC;

Query started at 07:56:14 PM India Standard Time; Query executed in 0.35 m

toc  pat_cnt  clm_cnt
0                None   650628  6470039
1          Commercial   578524  4154567
2        Medicare FFS   251832  1967038
3  Medicare Advantage    86039   795614
4              Others    96607   371882
5            Medicaid    62655   354965

In [37]:
%%read_sql
select count(*) as counts,
       count(distinct physician_npi) as cnts_physician,
       count(distinct patient_id) as patient_id,
       COUNT(DISTINCT CLAIM_NUMBER) AS CLM_CNT  
       from EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_TOC_sr

Query started at 07:56:38 PM India Standard Time; Query executed in 0.13 m

counts  cnts_physician  patient_id  clm_cnt
0  64367849          436832      650628  6470039

## Removing medicare ffs

In [38]:
%%read_sql
CREATE OR REPLACE TABLE EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_NONMED_sr as
     SELECT DISTINCT *
       FROM EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_TOC_sr
      WHERE lower(TOC) <> 'medicare ffs'
            OR lower(TOC) is null;

Query started at 07:56:56 PM India Standard Time; Query executed in 0.48 m

status
0  Table EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER...

In [44]:
%%read_sql
select count(*) as counts,
       count(distinct Physician_npi) as cnts_physician,
       count(distinct patient_id) as patient_count,
       count(distinct claim_number) as claim_count
       from EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_NONMED_sr

Query started at 06:21:21 PM India Standard Time; Query executed in 0.08 m

counts  cnts_physician  patient_count  claim_count
0  28787445          410561         647314      5669099

In [39]:
%%read_sql
    SELECT TOC,
           COUNT(DISTINCT PATIENT_ID) AS PAT_CNT,
           COUNT(DISTINCT CLAIM_NUMBER) AS CLM_CNT
      FROM EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_NONMED_sr
     GROUP BY TOC
     ORDER BY CLM_CNT DESC;

Query started at 07:57:57 PM India Standard Time; Query executed in 0.49 m

toc  pat_cnt  clm_cnt
0                None   650628  6470039
1          Commercial   578524  4154567
2  Medicare Advantage    86039   795614
3              Others    96607   371882
4            Medicaid    62655   354965

## NDC Filter

In [40]:
%%read_sql
CREATE OR REPLACE TABLE EYEPOINT_CATARACT_MED_NDC_PROV_NOR_NDC_Filter_sr AS
      SELECT A.PATIENT_ID,
             A.CLAIM_NUMBER,
             A.CATARACT_DATE,
             A.PATIENT_MIN_DATE,
             A.Physician_NPI,
             A.NDC,
             A.DRUG_DATE,
             A.TYPE_COVERAGE,
             A.TOC,
             A.PAYER_ID,
             A.name,
             A.FINGERTIP_PARENT_ID,
             A.FINGERTIP_PARENT_NAME,
             A.FINGERTIP_PAYER_ID,
             A.FINGERTIP_PAYER_NAME,
             B.PRODNME,
             B.GENNME,
             B.DRUG_TYPE,
             B.gen_bran
        FROM EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_NONMED_sr A
             JOIN ep_drug_ref_table B
                  ON UPPER(A.NDC) = UPPER(B.CODE)
       UNION
      SELECT A.PATIENT_ID,
             A.CLAIM_NUMBER,
             A.CATARACT_DATE,
             A.PATIENT_MIN_DATE,
             A.Physician_NPI,
             A.PROCEDURE,
             A.DRUG_DATE,
             A.TYPE_COVERAGE,
             A.TOC,
             A.PAYER_ID,
             A.name,
             A.FINGERTIP_PARENT_ID,
             A.FINGERTIP_PARENT_NAME,
             A.FINGERTIP_PAYER_ID,
             A.FINGERTIP_PAYER_NAME,
             B.PRODNME,
             B.GENNME,
             B.DRUG_TYPE,
             B.gen_bran
        FROM EYEPOINT_CATARACT_MED_NDC_PROV_NOR_PAYER_NONMED_sr A
             JOIN ep_drug_ref_table B
                  ON UPPER(A.PROCEDURE) = UPPER(B.CODE);

Query started at 07:58:30 PM India Standard Time; Query executed in 0.46 m

status
0  Table EYEPOINT_CATARACT_MED_NDC_PROV_NOR_NDC_F...

In [46]:
%%read_sql
select count(*) as counts,
       count(distinct Physician_npi) as cnts_Physician,
       count(distinct patient_id) as patient_id,
       count(distinct claim_number) as cnts_claims
       from EYEPOINT_CATARACT_MED_NDC_PROV_NOR_NDC_Filter_sr

Query started at 08:07:32 PM India Standard Time; Query executed in 0.05 m

counts  cnts_physician  patient_id  cnts_claims
0  722767           27025       81425       139595

# Raven Pharmacy

Getting all information from pharmacy including payer info fro 1 month before and 3 months after the first cataract surgery

In [51]:
%%read_sql
CREATE or replace TABLE EYEPOINT_CATARACT_Rx_NDC_SR_1 AS
      SELECT A.Physician_npi as Provider_id ,
             A.PATIENT_ID,
             B.CLAIM_NUMBER,
             A.YEAR_OF_SERVICE AS CATARACT_DATE,
             A.patient_min_date,
             B.Prescriber_id, 
             B.PRODUCT_OR_SERVICE_ID AS NDC,
             B.DATE_OF_SERVICE AS DRUG_DATE,
             B.RESPONSE_CODE,
             B.TYPE_OF_PAYMENT,
             B.PAYER_ID,
             B.PLAN_NAME,
             B.plan_type,
             B.INFERRED_PAYER_1,
             B.INFERRED_PAYER_2,
             B.INFERRED_PAYER_3,
             coalesce(B.INFERRED_PAYER_1,B.INFERRED_PAYER_2,B.INFERRED_PAYER_3) as INFERRED_PAYER_ID
        FROM EYEPOINT_CATARACT_MIN_DATE_nik_3_sr A
             LEFT JOIN RWD_DB.RWD.RAVEN_PHARMACY B
                       ON A.PATIENT_ID = case when B.ENCRYPTED_KEY_2 is null then B.ENCRYPTED_KEY_1 else B.ENCRYPTED_KEY_1||B.ENCRYPTED_KEY_2 end
                          AND UPPER(RESPONSE_CODE) = 'P'
                          AND DATEDIFF(day,patient_min_date,B.DATE_OF_SERVICE) between -30 and 90;

Query started at 08:07:50 PM India Standard Time; Query executed in 15.15 m

status
0  Table EYEPOINT_CATARACT_RX_NDC_SR successfully...

In [48]:
%%read_sql
select count(*) as counts,
       count(distinct provider_id) as cnts_providers,
       count(distinct Prescriber_id) as cnts_prescribers,
       count(distinct patient_id) as patient_counts,
       count(distinct claim_number) as cnts_claims
       from EYEPOINT_CATARACT_Rx_NDC_SR

Query started at 08:10:25 PM India Standard Time; Query executed in 0.10 m

counts  cnts_providers  cnts_prescribers  patient_counts  cnts_claims
0  9131353           19721            274137          650628      3758196

## TOC

In [53]:
%%read_sql
CREATE OR REPLACE TABLE EYEPOINT_CATARACT_Rx_NDC_SR_TOC AS
      SELECT A.*,
             C.MAP1 as TOC
        FROM EYEPOINT_CATARACT_Rx_NDC_SR A
             LEFT Join project_analytics.Pfizer.Plan_Type_Mapping_Options C
                    on A.TYPE_OF_PAYMENT = C.TYPE_OF_COVERAGE;
      

Query started at 08:27:11 PM India Standard Time; Query executed in 0.19 m

status
0  Table EYEPOINT_CATARACT_RX_NDC_SR_TOC successf...

In [54]:
%%read_sql
    SELECT TOC,
           COUNT(DISTINCT PATIENT_ID) AS PAT_CNT,
           COUNT(DISTINCT CLAIM_NUMBER) AS CLM_CNT
      FROM EYEPOINT_CATARACT_RX_NDC_SR_TOC 
     GROUP BY TOC
     ORDER BY CLM_CNT DESC;

Query started at 08:29:18 PM India Standard Time; Query executed in 0.11 m

toc  pat_cnt  clm_cnt
0                None   584752  2626075
1  Medicare Advantage    47268   644685
2          Commercial    37400   384679
3            Medicaid     4848    35785
4    Managed Medicaid     1870    34763
5              Others     5298    25770
6        Medicare FFS     2743     6438
7                CASH        1        1

## Removing medicare ffs

In [56]:
%%read_sql
CREATE OR REPLACE TABLE EYEPOINT_CATARACT_RX_NDC_SR_TOC_NONMED as
     SELECT DISTINCT *
       FROM EYEPOINT_CATARACT_RX_NDC_SR_TOC 
      WHERE lower(TOC) <> 'medicare ffs'
            OR lower(TOC) is null;

Query started at 08:32:09 PM India Standard Time; Query executed in 0.29 m

status
0  Table EYEPOINT_CATARACT_RX_NDC_SR_TOC_NONMED s...

**TOC NONE VALUES*** should we remove these values?

**ANSWER_chin** we should not remove here as we need all the claims here except medicare ffs

In [57]:
%%read_sql
    SELECT COUNT(DISTINCT PATIENT_ID) AS PAT_CNT,
           COUNT(DISTINCT CLAIM_NUMBER) AS CLM_CNT
      FROM EYEPOINT_CATARACT_RX_NDC_SR_TOC_NONMED 
   

Query started at 08:33:11 PM India Standard Time; Query executed in 0.11 m

pat_cnt  clm_cnt
0   650175  3751758

In [58]:
%%read_sql
    SELECT TOC,
           COUNT(DISTINCT PATIENT_ID) AS PAT_CNT,
           COUNT(DISTINCT CLAIM_NUMBER) AS CLM_CNT
      FROM EYEPOINT_CATARACT_RX_NDC_SR_TOC_NONMED
     GROUP BY TOC
     ORDER BY CLM_CNT DESC;

Query started at 08:33:32 PM India Standard Time; Query executed in 0.10 m

toc  pat_cnt  clm_cnt
0                None   584752  2626075
1  Medicare Advantage    47268   644685
2          Commercial    37400   384679
3            Medicaid     4848    35785
4    Managed Medicaid     1870    34763
5              Others     5298    25770
6                CASH        1        1

## NDC Filter

In [59]:
%%read_sql
CREATE OR REPLACE TABLE EYEPOINT_CATARACT_RX_NDC_SR_NDC_Filter AS
      SELECT A.*,
             B.PRODNME,
             B.GENNME,
             B.DRUG_TYPE,
             B.gen_bran
        FROM EYEPOINT_CATARACT_RX_NDC_SR_TOC_NONMED A
             JOIN ep_drug_ref_table B
                  ON UPPER(A.NDC) = UPPER(B.CODE);

Query started at 08:37:49 PM India Standard Time; Query executed in 0.13 m

status
0  Table EYEPOINT_CATARACT_RX_NDC_SR_NDC_FILTER s...

In [6]:
%%read_sql
select count(*) as counts,
       count(distinct prescriber_id) as cnts_prescriber_id,
       count(distinct provider_id) as cnts_provider_id,
       count(distinct patient_id) as patient_count,
        count(distinct claim_number) as calim_count
       from EYEPOINT_CATARACT_RX_NDC_SR_NDC_Filter

Query started at 01:34:44 PM India Standard Time; Query executed in 0.06 m

counts  cnts_prescriber_id  cnts_provider_id  patient_count  calim_count
0  1020478               35983             12663         157472       630711

# Raven Med + Rx

In [42]:
%%read_sql
CREATE OR REPLACE TABLE EYEPOINT_CATARACT_RX_MED_NDC_SR_1 AS
      SELECT DISTINCT 'raven_med' as SOURCE,
                      PATIENT_ID,
                      CLAIM_NUMBER,
                      Physician_npi,
                      NDC,
                      TOC,
                      CATARACT_DATE,
                      PATIENT_MIN_DATE,
                      DRUG_DATE,
                      PAYER_ID,
                      FINGERTIP_PAYER_ID,
                      FINGERTIP_PAYER_NAME,
                      DRUG_TYPE,
                      GEN_BRAN
        FROM EYEPOINT_CATARACT_MED_NDC_PROV_NOR_NDC_Filter_sr
       UNION
      SELECT DISTINCT 'raven_rx' as SOURCE,
                      A.PATIENT_ID, 
                      A.CLAIM_NUMBER, 
                      A.Prescriber_id as Physician_npi,
                      A.NDC, 
                      A.TOC,
                      A.CATARACT_DATE,
                      A.PATIENT_MIN_DATE,
                      A.DRUG_DATE,
                      A.PAYER_ID,
                      A.INFERRED_PAYER_ID,
                      B.PAYER_NAME,
                      A.DRUG_TYPE,
                      A.GEN_BRAN
        FROM EYEPOINT_CATARACT_RX_NDC_SR_NDC_Filter A
             LEFT JOIN rwd_db.rwd.third_party_fingertip_payers B
                       ON A.INFERRED_PAYER_ID = B.ID;

Query started at 08:00:50 PM India Standard Time; Query executed in 0.20 m

status
0  Table EYEPOINT_CATARACT_RX_MED_NDC_SR_1 succes...

In [45]:
%%read_sql
select source,
       count(*) as counts,
       count(distinct physician_npi) as cnts_physician_npi,
       count(distinct patient_id) as patient_count,
       count(distinct claim_number) as calim_count
  from EYEPOINT_CATARACT_RX_MED_NDC_SR 
  Group by 1
    

Query started at 08:05:05 PM India Standard Time; Query executed in 0.12 m

source  counts  cnts_physician_npi  patient_count  calim_count
0  raven_med  389388               24553          74498       123807
1   raven_rx  996601               35983         157472       630711

In [43]:
%%read_sql
select count(*) as counts,
    count(distinct physician_npi) as cnts_physician_npi,
       count(distinct patient_id) as patient_count,
        count(distinct claim_number) as calim_count
       from EYEPOINT_CATARACT_RX_MED_NDC_SR_1 
      

Query started at 08:02:27 PM India Standard Time; Query executed in 0.09 m

counts  cnts_physician_npi  patient_count  calim_count
0  1719049               57022         218327       770306

In [44]:
%%read_sql
select source,
       count(*) as counts,
       count(distinct physician_npi) as cnts_physician_npi,
       count(distinct patient_id) as patient_count,
       count(distinct claim_number) as calim_count
  from EYEPOINT_CATARACT_RX_MED_NDC_SR_1 
  Group by 1

Query started at 08:03:40 PM India Standard Time; Query executed in 0.08 m

source  counts  cnts_physician_npi  patient_count  calim_count
0  raven_med  722448               27025          81425       139595
1   raven_rx  996601               35983         157472       630711

In [3]:
%%read_sql

SELECT count(distinct physician_npi)
  FROM EYEPOINT_CATARACT_RX_MED_NDC_SR_1 
WHERE physician_npi in (SELECT distinct physician_npi
                          FROM Eyepoint_cataract_raven_phy)

Query started at 08:36:09 PM India Standard TimeInitiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
; Query executed in 0.32 m

COUNT(DISTINCT PHYSICIAN_NPI)
0                          11850

In [ ]:
---completed till here----

# Adding single physician specialities

In [4]:
%%read_sql

Create or replace table EYEPOINT_CATARACT_RX_MED_NDC_SR_phy_spec as 
    SELECT distinct A.*,
           coalesce(B.specialty_fin,C.SPECIALTY,D.collapsed_specialization) as physician_specialty
      From EYEPOINT_CATARACT_RX_MED_NDC_SR_1 A
           LEFT JOIN EYEPOINT_SPECIALTY_STACKED_RAW_v2_grouped B
                 ON A.physician_npi = cast(B.npi as varchar)
           LEFT JOIN Physician_Speciality C
                 ON A.physician_npi = cast(C.npi as varchar)
           LEFT JOIN eyepoint_gen_coalesce_specialty D
                 ON A.physician_npi = cast(D.npi as varchar)

Query started at 02:20:56 PM India Standard Time; Query executed in 0.37 m

status
0  Table EYEPOINT_CATARACT_RX_MED_NDC_SR_PHY_SPEC...

In [6]:
%%read_sql

SELECT * FROM EYEPOINT_CATARACT_RX_MED_NDC_SR_phy_spec 
WHERE physician_specialty is not null LIMIT 10

Query started at 02:22:39 PM India Standard Time; Query executed in 0.05 m

source                                         patient_id  \
0  raven_med  Z/f3domI21PuAlj/V/d9/LEA4mdfAmDowp95eI0LQtg=D1...   
1  raven_med  0n8V3EYo1e4XArDOVNvqECT01R7JvqfqmFD5dJSAG2c=n3...   
2  raven_med  0n8V3EYo1e4XArDOVNvqECT01R7JvqfqmFD5dJSAG2c=n3...   
3  raven_med  t0rZ4lOwiFE1VHKs7h9ACrGEMpZvlCo60gbs+wcYN6A=82...   
4  raven_med  Z/f3domI21PuAlj/V/d9/LEA4mdfAmDowp95eI0LQtg=D1...   
5   raven_rx  rVynTMfanaEWWyiPzs/2eY/39cCFd5GNc+fnsyMKM1Q=vL...   
6  raven_med  6VqfQ7d/JrXA3ySnhd9XDOOWpnZEQdWO7hRhz2qOi1c=lo...   
7  raven_med  JBHB0gD40zLZ+Lyjjh6vmPlQTcu/KJ7bt8nSRBl3fT4=Sr...   
8   raven_rx  EoZabgAsd6YNywWJqpXaHRi3KVSPuWGBL41HjKc8erI=vV...   
9  raven_med  UUTuSSiQpITXdt3l9W93LRY5mMu2vtuXFYcQ9eOUtUE=hE...   

            claim_number physician_npi          ndc         toc cataract_date  \
0     282966000000129342    1508021494        J1100  Commercial    2017-10-27   
1     288250000000206180    1508027020        J1885  Commercial    2018-01-17   
2  con_EI020718246158367    1508027020        J1885        None    2018-01-17   
3  con_EI050418271554869    1508021494  63323016501        None    2018-04-27   
4     286441000000023124    1508021494        J1100  Commercial    2017-10-27   
5     281634000000097481    1508021494  61314063705        None    2017-10-06   
6  con_EI062817281424816    1508021494  00641036725        None    2017-06-23   
7     290314000000130236    1508021494        J1100        None    2018-03-02   
8     259784000004334682    1508021494  17478020911        None    2017-06-02   
9  con_EI120517221731966    1508022286        J1100        None    2017-11-30   

  patient_min_date   drug_date payer_id fingertip_payer_id  \
0       2017-10-27  2017-10-27    00303               None   
1       2018-01-17  2018-01-28     3550               None   
2       2018-01-17  2018-01-28     None               None   
3       2018-04-06  2018-04-27     None               None   
4       2017-10-27  2017-11-10     3520               None   
5       2017-10-06  2017-10-14     None                677   
6       2017-06-23  2017-06-23     None               None   
7       2018-03-02  2018-03-09     None               None   
8       2017-06-02  2017-05-03     None                 89   
9       2017-11-21  2017-11-30     None               None   

  fingertip_payer_name drug_type  gen_bran physician_specialty  
0                 None   STEROID  Generics       Ophthalmology  
1                 None     NSAID  Generics  Emergency Medicine  
2                 None     NSAID  Generics  Emergency Medicine  
3                 None   STEROID  Generics       Ophthalmology  
4                 None   STEROID  Generics       Ophthalmology  
5              OptumRx   STEROID  Generics       Ophthalmology  
6                 None   STEROID  Generics       Ophthalmology  
7                 None   STEROID  Generics       Ophthalmology  
8         CVS/Caremark     NSAID  Generics       Ophthalmology  
9                 None   STEROID  Generics       Ophthalmology

# Creating Output (physician level)

## Creating flags for required condition

In [4]:
%%read_sql

CREATE OR REPLACE TABLE EYEPOINT_CATARACT_RX_MED_NDC_SR_indicator AS
    SELECT distinct physician_npi,
           patient_id,
           case when lower(drug_type) = 'omi' then 1 else 0 end as ind_omidria,
           case when lower(drug_type) = 'steroid' then 1 else 0 end as ind_steroid,
           case when lower(drug_type) = 'nsaid' then 1 else 0 end as ind_nsaid         
    FROM EYEPOINT_CATARACT_RX_MED_NDC_SR_phy_spec
    WHERE physician_specialty not in (select specialty from Specialities_Eyepoint where remove_flag='1')
      AND physician_npi is not null
      AND length(physician_npi) = 10

Query started at 02:33:32 PM India Standard Time; Query executed in 0.09 m

status
0  Table EYEPOINT_CATARACT_RX_MED_NDC_SR_INDICATO...

In [5]:
%%read_sql

SELECT * FROM EYEPOINT_CATARACT_RX_MED_NDC_SR_indicator 
ORDER BY patient_id LIMIT 10


Query started at 02:33:40 PM India Standard Time; Query executed in 0.05 m

physician_npi                                         patient_id  \
0    1366425704  +++LqZPeSkoH2+AR2w9+1A/fmnnFb7yZ83NgMN0Gnlo=Wz...   
1    1053487736  ++0pNgvDhHHOxea8etHqk991ZfDOqtyOBRo+W5IVad0=s/...   
2    1053487736  ++0pNgvDhHHOxea8etHqk991ZfDOqtyOBRo+W5IVad0=s/...   
3    1336168624  ++1tvWScpvVuD5FRElM4vpUZRIDEexWF8PShXJCQY30=x9...   
4    1154401842  ++2Q9p5DObjl0e+BBcXhPIVjNMJLa5vtoG0Cnx/cw9I=z0...   
5    1063592723  ++2Q9p5DObjl0e+BBcXhPIVjNMJLa5vtoG0Cnx/cw9I=z0...   
6    1063592723  ++2Q9p5DObjl0e+BBcXhPIVjNMJLa5vtoG0Cnx/cw9I=z0...   
7    1336309723  ++2sYPSVyDl8+PHaEbd+ilPX++ShCReu4xbt9qzkjpw=xN...   
8    1356337562  ++2sYPSVyDl8+PHaEbd+ilPX++ShCReu4xbt9qzkjpw=xN...   
9    1336309723  ++2sYPSVyDl8+PHaEbd+ilPX++ShCReu4xbt9qzkjpw=xN...   

   ind_omidria  ind_steroid  ind_nsaid  
0            0            1          0  
1            0            1          0  
2            0            0          1  
3            0            1          0  
4            0            1          0  
5            0            1          0  
6            0            0          1  
7            0            1          0  
8            0            1          0  
9            0            0          1

In [6]:
%%read_sql

SELECT count(*), count(distinct patient_id), count(distinct physician_npi) FROM EYEPOINT_CATARACT_RX_MED_NDC_SR_indicator LIMIT 10

Query started at 02:33:48 PM India Standard Time; Query executed in 0.04 m

COUNT(*)  COUNT(DISTINCT PATIENT_ID)  COUNT(DISTINCT PHYSICIAN_NPI)
0    358275                      214319                          55386

In [7]:
%%read_sql

CREATE OR REPLACE TABLE EYEPOINT_CATARACT_RX_MED_NDC_SR_drg_cnts AS
    SELECT distinct physician_npi,
           count(distinct patient_id) as total_rx_pat_cnt,
           count(distinct case when ind_nsaid = 1 then patient_id else null end) as nsaid_pat_cnt,
           count(distinct case when ind_steroid = 1 then patient_id else null end) as steroid_pat_cnt,
           count(distinct case when ind_omidria = 1 then patient_id else null end) as omi_pat_cnt
    FROM EYEPOINT_CATARACT_RX_MED_NDC_SR_indicator
    Group by 1
 

Query started at 02:34:07 PM India Standard Time; Query executed in 0.07 m

status
0  Table EYEPOINT_CATARACT_RX_MED_NDC_SR_DRG_CNTS...

In [8]:
%%read_sql

SELECT count(distinct physician_npi)
FROM EYEPOINT_CATARACT_RX_MED_NDC_SR_drg_cnts

Query started at 02:34:18 PM India Standard Time; Query executed in 0.12 m

COUNT(DISTINCT PHYSICIAN_NPI)
0                          55386

In [9]:
%%read_sql

SELECT * FROM EYEPOINT_CATARACT_RX_MED_NDC_SR_drg_cnts LIMIT 10

Query started at 02:34:30 PM India Standard Time; Query executed in 0.08 m

physician_npi  total_rx_pat_cnt  nsaid_pat_cnt  steroid_pat_cnt  omi_pat_cnt
0    1215998315                 1              0                1            0
1    1184659112                 2              0                2            0
2    1952631632                20             19               19            0
3    1780656298                 1              1                0            0
4    1326044041                 2              0                2            0
5    1477723187                 2              0                2            0
6    1467966473                 1              0                1            0
7    1790978492                 1              0                1            0
8    1912952953                 1              0                1            0
9    1356348528                 2              0                2            0

## Getting physician and facility details corresponding to physicians

In [33]:
%%read_sql

CREATE OR REPLACE TABLE EYEPOINT_CATARACT_RX_MED_NDC_SR_drg_cnts_w_det AS
    SELECT distinct A.*,
           B.physician_state
          ,B.FACILITY_HB_ID
          ,B.LIST_OF_NPIS
          ,B.FACILITY_NAME
          ,B.FACILITY_ADDRESS
          ,B.FACILITY_CITY
          ,B.FACILITY_STATE
          ,left(B.FACILITY_ZIP,5) as facility_zip
     FROM EYEPOINT_CATARACT_RX_MED_NDC_SR_drg_cnts A
          LEFT JOIN eyepoint_cataract_fac_phy_dexycu_hb_rx_7 B
                 ON A.physician_npi = B.physician_npi

Query started at 04:01:10 PM India Standard Time; Query executed in 0.09 m

status
0  Table EYEPOINT_CATARACT_RX_MED_NDC_SR_DRG_CNTS...

## Rolling upto facilities

In [37]:
%%read_sql

CREATE OR REPLACE TABLE EYEPOINT_CATARACT_RX_MED_NDC_SR_drg_cnts_w_det_fac AS
    SELECT distinct 
           facility_hb_id
          ,LIST_OF_NPIS
          ,FACILITY_NAME
          ,FACILITY_ADDRESS
          ,FACILITY_CITY
          ,FACILITY_STATE
          ,FACILITY_ZIP
          ,sum(total_rx_pat_cnt) as total_rx_pat_cnt
          ,sum(nsaid_pat_cnt) as nsaid_pat_cnt
          ,sum(steroid_pat_cnt) as steroid_pat_cnt
          ,sum(omi_pat_cnt) as omi_pat_cnt
     FROM EYEPOINT_CATARACT_RX_MED_NDC_SR_drg_cnts_w_det
     WHERE physician_state not in ('GU','MP','VI','AE','AP','AS') 
     and facility_state not in ('GU','MP','VI','AE','AP','AS')
     Group by 1,2,3,4,5,6,7
          

Query started at 04:28:16 PM India Standard Time; Query executed in 0.07 m

status
0  Table EYEPOINT_CATARACT_RX_MED_NDC_SR_DRG_CNTS...

In [38]:
%%read_sql

SELECT count(*), count(distinct facility_hb_id)
  FROM EYEPOINT_CATARACT_RX_MED_NDC_SR_drg_cnts_w_det_fac

Query started at 04:28:23 PM India Standard Time; Query executed in 0.24 m

COUNT(*)  COUNT(DISTINCT FACILITY_HB_ID)
0     20548                           20548

## Joining in with facilities from cataract 

In [ ]:
%%read_sql

Create or replace table tab2_comb_cataract_rx as 
     SELECT 

# Adding facility mastering from Isha

In [12]:
%%read_sql

Create or replace table EYEPOINT_CATARACT_RX_MED_NDC_SR_2 as 
    SELECT A.SOURCE,
           A.PATIENT_ID,
           A.CLAIM_NUMBER,
           A.Physician_npi,
           A.physician_specialty,
           A.NDC,
           A.TOC,
           A.CATARACT_DATE,
           A.PATIENT_MIN_DATE,
           A.DRUG_DATE,
           A.PAYER_ID,
           A.FINGERTIP_PAYER_ID,
           A.FINGERTIP_PAYER_NAME,
           A.DRUG_TYPE,
           A.GEN_BRAN
          ,B.physician_state
          ,B.FACILITY_HB_ID
          ,B.LIST_OF_NPIS
          ,B.FACILITY_NAME
          ,B.FACILITY_ADDRESS
          ,B.FACILITY_CITY
          ,B.FACILITY_STATE
          ,B.FACILITY_ZIP
          ,B.PARENT_NAME
          ,B.PARENT_HB_ID
          ,B.PARENT_ADDRESS
          ,B.PARENT_CITY
          ,B.PARENT_STATE
          ,B.PARENT_ZIP
     FROM EYEPOINT_CATARACT_RX_MED_NDC_SR_phy_spec A
          LEFT JOIN eyepoint_cataract_fac_phy_dexycu_hb_rx_6 B
                 ON A.claim_number = B.claim_number
                AND A.patient_id = B.patient_id
                AND A.physician_npi = B.physician_npi

Query started at 02:44:33 PM India Standard Time; Query executed in 0.24 m

status
0  Table EYEPOINT_CATARACT_RX_MED_NDC_SR_2 succes...

In [13]:
%%read_sql

SELECT count(*)
FROM EYEPOINT_CATARACT_RX_MED_NDC_SR_2

Query started at 02:44:53 PM India Standard Time; Query executed in 0.05 m

COUNT(*)
0   1719049

## Creating physician, fac, parent det table

# Creating output (facility level) -- already incld fac from surgeries

## Creating flags for required condition

In [23]:
%%read_sql

CREATE OR REPLACE TABLE EYEPOINT_CATARACT_RX_MED_NDC_SR_indicator_fac AS
    SELECT distinct facility_hb_id,
           patient_id,
           case when lower(drug_type) = 'omi' then 1 else 0 end as ind_omidria,
           case when lower(drug_type) = 'steroid' then 1 else 0 end as ind_steroid,
           case when lower(drug_type) = 'nsaid' then 1 else 0 end as ind_nsaid         
    FROM EYEPOINT_CATARACT_RX_MED_NDC_SR_2
    WHERE facility_state not in ('GU','MP','VI','AE','AP','AS')
      AND physician_specialty not in (select specialty from Specialities_Eyepoint where remove_flag='1')
      AND physician_npi is not null
      AND length(physician_npi) = 10

Query started at 03:47:47 PM India Standard Time; Query executed in 0.22 m

status
0  Table EYEPOINT_CATARACT_RX_MED_NDC_SR_INDICATO...

In [24]:
%%read_sql

SELECT * FROM EYEPOINT_CATARACT_RX_MED_NDC_SR_indicator_fac
ORDER BY patient_id LIMIT 10


Query started at 03:48:03 PM India Standard Time; Query executed in 0.06 m

facility_hb_id                                         patient_id  \
0  ASC:39C0001108  +++LqZPeSkoH2+AR2w9+1A/fmnnFb7yZ83NgMN0Gnlo=Wz...   
1  ASC:51C0001011  ++0pNgvDhHHOxea8etHqk991ZfDOqtyOBRo+W5IVad0=s/...   
2  ASC:51C0001011  ++0pNgvDhHHOxea8etHqk991ZfDOqtyOBRo+W5IVad0=s/...   
3   PG:9335238732  ++1tvWScpvVuD5FRElM4vpUZRIDEexWF8PShXJCQY30=x9...   
4  ASC:39C0001177  ++2Q9p5DObjl0e+BBcXhPIVjNMJLa5vtoG0Cnx/cw9I=z0...   
5  ASC:39C0001177  ++2Q9p5DObjl0e+BBcXhPIVjNMJLa5vtoG0Cnx/cw9I=z0...   
6  ASC:33C0001045  ++2sYPSVyDl8+PHaEbd+ilPX++ShCReu4xbt9qzkjpw=xN...   
7  ASC:33C0001045  ++2sYPSVyDl8+PHaEbd+ilPX++ShCReu4xbt9qzkjpw=xN...   
8   PG:6406754431  ++51C75AQR+PoGnSOcbzviLgIelleksAN5hLOKEbj+E=GO...   
9  ASC:13C0001032  ++7fZ4vXvBx6t2nPgNzEi3MG7qEmrd0rowAB+BdHRLw=Sx...   

   ind_omidria  ind_steroid  ind_nsaid  
0            0            1          0  
1            0            1          0  
2            0            0          1  
3            0            1          0  
4            0            0          1  
5            0            1          0  
6            0            1          0  
7            0            0          1  
8            0            1          0  
9            0            0          1

In [26]:
%%read_sql

SELECT count(*), count(distinct patient_id), count(distinct facility_hb_id) FROM EYEPOINT_CATARACT_RX_MED_NDC_SR_indicator_fac LIMIT 10

Query started at 03:48:45 PM India Standard Time; Query executed in 0.09 m

COUNT(*)  COUNT(DISTINCT PATIENT_ID)  COUNT(DISTINCT FACILITY_HB_ID)
0    282492                      183621                            5123

In [27]:
%%read_sql

CREATE OR REPLACE TABLE EYEPOINT_CATARACT_RX_MED_NDC_SR_drg_cnts_fac AS
    SELECT distinct facility_hb_id,
           count(distinct patient_id) as total_rx_pat_cnt,
           count(distinct case when ind_nsaid = 1 then patient_id else null end) as nsaid_pat_cnt,
           count(distinct case when ind_steroid = 1 then patient_id else null end) as steroid_pat_cnt,
           count(distinct case when ind_omidria = 1 then patient_id else null end) as omi_pat_cnt
    FROM EYEPOINT_CATARACT_RX_MED_NDC_SR_indicator_fac
    Group by 1
 

Query started at 03:49:23 PM India Standard Time; Query executed in 0.12 m

status
0  Table EYEPOINT_CATARACT_RX_MED_NDC_SR_DRG_CNTS...

In [30]:
%%read_sql

SELECT * FROM EYEPOINT_CATARACT_RX_MED_NDC_SR_drg_cnts_fac LIMIT 10

Query started at 03:50:25 PM India Standard Time; Query executed in 0.05 m

facility_hb_id  total_rx_pat_cnt  nsaid_pat_cnt  steroid_pat_cnt  \
0  ASC:23C0001047                23              2               23   
1  ASC:31C0001136                18              6               16   
2   PG:9739392457                13              1               13   
3        H:140015                 2              0                2   
4        H:270004                15              9               14   
5  ASC:21C0001145               227            127              223   
6   PG:7517933187                 1              1                1   
7   PG:9931238094                 2              2                2   
8   PG:6002728532                 5              2                4   
9   PG:2961695754                 2              2                2   

   omi_pat_cnt  
0            0  
1            0  
2            0  
3            0  
4            0  
5            0  
6            0  
7            0  
8            0  
9            0

# Omidria

PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.EYEPOINT_CATARACT_CMS_OP_PROC  
PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.EYEPOINT_CATARACT_CMS_ASC_PROC

In [ ]:
%%read_sql cms_omidria_op -d

CREATE OR REPLACE table EYEPOINT_CATARACT_CMS_OP_PROC_1 as
     SELECT 
SELECT distinct facility_npi, count(distinct patient_id) as pat_cnt
  from PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.EYEPOINT_CATARACT_CMS_OP_PROC
 group by facility_npi